В этом файле я буду создавать миди файлы с партиями разных инструментов в разных произведениях. Количество файлов, полученных из 1 файла, совпадает с количеством инструментов, звучащих в этом файле

In [2]:
import os
import pretty_midi
from collections import defaultdict

def get_instrument_name(program_number):
    general_midi_instruments = {
        0: 'Acoustic Grand Piano',
        1: 'Bright Acoustic Piano',
        2: 'Electric Grand Piano',
        3: 'Honky-tonk Piano',
        4: 'Electric Piano 1',
        5: 'Electric Piano 2',
        6: 'Harpsichord',
        7: 'Clavinet',
        8: 'Celesta',
        9: 'Glockenspiel',
        10: 'Music Box',
        11: 'Vibraphone',
        12: 'Marimba',
        13: 'Xylophone',
        14: 'Tubular Bells',
        15: 'Dulcimer',
        16: 'Drawbar Organ',
        17: 'Percussive Organ',
        18: 'Rock Organ',
        19: 'Church Organ',
        20: 'Reed Organ',
        21: 'Accordion',
        22: 'Harmonica',
        23: 'Tango Accordion',
        24: 'Acoustic Guitar (nylon)',
        25: 'Acoustic Guitar (steel)',
        26: 'Electric Guitar (jazz)',
        27: 'Electric Guitar (clean)',
        28: 'Electric Guitar (muted)',
        29: 'Overdriven Guitar',
        30: 'Distortion Guitar',
        31: 'Guitar Harmonics',
        32: 'Acoustic Bass',
        33: 'Electric Bass (finger)',
        34: 'Electric Bass (pick)',
        35: 'Fretless Bass',
        36: 'Slap Bass 1',
        37: 'Slap Bass 2',
        38: 'Synth Bass 1',
        39: 'Synth Bass 2',
        40: 'Violin',
        41: 'Viola',
        42: 'Cello',
        43: 'Contrabass',
        44: 'Tremolo Strings',
        45: 'Pizzicato Strings',
        46: 'Orchestral Harp',
        47: 'Timpani',
        48: 'String Ensemble 1',
        49: 'String Ensemble 2',
        50: 'Synth Strings 1',
        51: 'Synth Strings 2',
        52: 'Choir Aahs',
        53: 'Voice Oohs',
        54: 'Synth Voice',
        55: 'Orchestra Hit',
        56: 'Trumpet',
        57: 'Trombone',
        58: 'Tuba',
        59: 'Muted Trumpet',
        60: 'French Horn',
        61: 'Brass Section',
        62: 'Synth Brass 1',
        63: 'Synth Brass 2',
        64: 'Soprano Sax',
        65: 'Alto Sax',
        66: 'Tenor Sax',
        67: 'Baritone Sax',
        68: 'Oboe',
        69: 'English Horn',
        70: 'Bassoon',
        71: 'Clarinet',
        72: 'Piccolo',
        73: 'Flute',
        74: 'Recorder',
        75: 'Pan Flute',
        76: 'Blown Bottle',
        77: 'Shakuhachi',
        78: 'Whistle',
        79: 'Ocarina',
        80: 'Lead 1 (square)',
        81: 'Lead 2 (sawtooth)',
        82: 'Lead 3 (calliope)',
        83: 'Lead 4 (chiff)',
        84: 'Lead 5 (charang)',
        85: 'Lead 6 (voice)',
        86: 'Lead 7 (fifths)',
        87: 'Lead 8 (bass + lead)',
        88: 'Pad 1 (new age)',
        89: 'Pad 2 (warm)',
        90: 'Pad 3 (polysynth)',
        91: 'Pad 4 (choir)',
        92: 'Pad 5 (bowed)',
        93: 'Pad 6 (metallic)',
        94: 'Pad 7 (halo)',
        95: 'Pad 8 (sweep)',
        96: 'FX 1 (rain)',
        97: 'FX 2 (soundtrack)',
        98: 'FX 3 (crystal)',
        99: 'FX 4 (atmosphere)',
        100: 'FX 5 (brightness)',
        101: 'FX 6 (goblins)',
        102: 'FX 7 (echoes)',
        103: 'FX 8 (sci-fi)',
        104: 'Sitar',
        105: 'Banjo',
        106: 'Shamisen',
        107: 'Koto',
        108: 'Kalimba',
        109: 'Bagpipe',
        110: 'Fiddle',
        111: 'Shanai',
        112: 'Tinkle Bell',
        113: 'Agogo',
        114: 'Steel Drums',
        115: 'Woodblock',
        116: 'Taiko Drum',
        117: 'Melodic Tom',
        118: 'Synth Drum',
        119: 'Reverse Cymbal',
        120: 'Guitar Fret Noise',
        121: 'Breath Noise',
        122: 'Seashore',
        123: 'Bird Tweet',
        124: 'Telephone Ring',
        125: 'Helicopter',
        126: 'Applause',
        127: 'Gunshot',
    }
    return general_midi_instruments.get(program_number, f'Unknown_Instrument_{program_number}')

def process_midi_file(midi_path, output_folder):
    try:
        # Загрузка MIDI-файла
        midi_data = pretty_midi.PrettyMIDI(midi_path)
        
        # Создание папки для выходных файлов
        os.makedirs(output_folder, exist_ok=True)

        # Обработка каждого инструмента в MIDI-файле
        for instrument in midi_data.instruments:
            program_number = instrument.program
            instrument_name = get_instrument_name(program_number)
            
            # Создаем безопасное имя файла
            safe_name = "".join(c if c.isalnum() or c in (' ', '_') else '_' for c in instrument_name)
            output_path = os.path.join(output_folder, f'{safe_name}.mid')
            
            # Если файл уже существует - загружаем его и добавляем ноты
            if os.path.exists(output_path):
                existing_midi = pretty_midi.PrettyMIDI(output_path)
                # Находим соответствующий инструмент или создаем новый
                found = False
                for existing_instr in existing_midi.instruments:
                    if existing_instr.program == program_number:
                        existing_instr.notes.extend(instrument.notes)
                        found = True
                        break
                
                if not found:
                    new_instrument = pretty_midi.Instrument(program=program_number)
                    new_instrument.notes = instrument.notes.copy()
                    existing_midi.instruments.append(new_instrument)
                
                existing_midi.write(output_path)
            else:
                # Создаем новый файл для инструмента
                new_midi = pretty_midi.PrettyMIDI()
                new_instrument = pretty_midi.Instrument(program=program_number)
                new_instrument.notes = instrument.notes.copy()
                new_midi.instruments.append(new_instrument)
                new_midi.write(output_path)
            
            print(f'Processed: {output_path}')

    except Exception as e:
        print(f"Ошибка при обработке файла {midi_path}: {e}")

def process_all_midi(input_folder, output_folder):
    if not os.path.exists(input_folder):
        print(f"Папка {input_folder} не существует!")
        return
    
    os.makedirs(output_folder, exist_ok=True)
    
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith('.mid'):
                midi_path = os.path.join(root, file)
                process_midi_file(midi_path, output_folder)

# Пример использования
midi_folder = 'lettersabc/a'
output_folder = os.path.join(midi_folder, 'instruments')
process_all_midi(midi_folder, output_folder)

Processed: lettersabc/a\instruments\Accordion.mid
Processed: lettersabc/a\instruments\Acoustic Bass.mid
Processed: lettersabc/a\instruments\Acoustic Grand Piano.mid
Processed: lettersabc/a\instruments\Acoustic Guitar _nylon_.mid
Processed: lettersabc/a\instruments\Bright Acoustic Piano.mid
Processed: lettersabc/a\instruments\Choir Aahs.mid
Processed: lettersabc/a\instruments\Clarinet.mid
Processed: lettersabc/a\instruments\Contrabass.mid
Processed: lettersabc/a\instruments\Electric Guitar _jazz_.mid
Processed: lettersabc/a\instruments\Electric Guitar _muted_.mid
Processed: lettersabc/a\instruments\French Horn.mid
Processed: lettersabc/a\instruments\Fretless Bass.mid
Processed: lettersabc/a\instruments\FX 4 _atmosphere_.mid
Processed: lettersabc/a\instruments\Kalimba.mid
Processed: lettersabc/a\instruments\Lead 1 _square_.mid
Processed: lettersabc/a\instruments\Overdriven Guitar.mid
Processed: lettersabc/a\instruments\Pad 7 _halo_.mid
Processed: lettersabc/a\instruments\Percussive Organ

KeyboardInterrupt: 

Успешно были получены файлы с партиями отдельных инструментов!